In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
import string
import re
from nltk.corpus import wordnet
import nltk
from langdetect import detect,LangDetectException
import random

nltk.download('omw-1.4')
import pandas as pd

df = pd.read_csv('./data/data_merged_all.csv', index_col=0)
df = df.dropna(subset=['text'])
df

[nltk_data] Downloading package omw-1.4 to /home/self-
[nltk_data]     ouss/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,title,text,stars,label,language
0,gAAAAABnUH4NgkTQ9eyZpcZr0JpVZKT2A_-T3BUnPBPZps...,Sauce vipo infecte piquante sans aucun goût ni...,2,0,NaN
1,gAAAAABnUH4NyGLVcUk0aT-6YySNpN-Atb905EoDedSWV2...,"La cuisine est médiocre, la sauce Vipo est san...",1,0,NaN
2,gAAAAABnUH4NDFwRDSycnVfF_qiCYlFhB60ebSobqCmoWh...,الاجواء و الاضاءة روعة\n النظافة في القمة\n ال...,5,1,NaN
3,gAAAAABnUH4NRqzylGPMu-ODANAHON7KKDiz4uJhnHJUfQ...,"J'ai vraiment aimé cette place, le service est...",4,1,"{""choices"":[""French"",""Darija""]}"
4,gAAAAABnUH4NiAgbqy6d_5sxsXSJ5Ly_TOLP2DMYqka9_4...,"Cozy place, good burgers for the price u pay, ...",4,1,English
...,...,...,...,...,...
8649,gAAAAABnUH4OYSdSUVcbwuETc4RHERBE7DzjpU4PnXqeVF...,Khalwiii chez fathii à essayer bn courage l'éq...,5,1,NaN
8650,gAAAAABnUH4Oqmg9xzNb66Ma5NaZjqa6Z6kqU73gO5NXOM...,"C'est un jolie endroit, cependant le service m...",2,0,NaN
8651,gAAAAABnUH4OTULlw_eOujx6OOScmxaSHf4IIeHfzDn_aY...,Un très beau salon bonne équipe et une pizza m...,3,1,French
8652,gAAAAABnUH4OJfeul6XUIDtGV42vbzUz--fOy2Jl4cmAtC...,Un bon endroit pour passé du temps avec des am...,4,1,French


In [2]:
df[df["text"].duplicated()]

,title,text,stars,label,language
218,gAAAAABnUH4NSDh9FhRcUByELL3CREN1Mc6YUR5xYVfb4b...,Excellent,5,1,NaN
318,gAAAAABnUH4Nd8SDUSucscHnkgeevU3pln6C5exDektcZI...,Pas mal,4,1,French
319,gAAAAABnUH4N2gCqnoKvkOkcYRnFtWL74KaYwhjhVVwbcW...,Bravo,5,1,NaN
324,gAAAAABnUH4NPgj1_a2zoTAmF6vEGFWEHjKoq3RJEWPocd...,Nul,1,0,NaN
343,gAAAAABnUH4Nn64V1V6axmR4bn1kuOO1zUB0w8CEoUn8Tf...,Pas mal,3,1,French
...,...,...,...,...,...
8606,gAAAAABnUH4OpuaZav4w2G4sl7ySQsMFMxxISJ8CCjfJax...,Goood,5,1,NaN
8611,gAAAAABnUH4ODJqbKFBzlFcIpbdSy3XEoQA64ZxUqf07Ai...,Bien,3,1,French
8641,gAAAAABnUH4OoLcq_FgiCqOZPUDAoYPhduGelxShZmCE5P...,Bon endroit,5,1,NaN
8643,gAAAAABnUH4O7I_3sFAt5_hVe_AhGIIERCCzwlOOWqmNvu...,Merci,5,1,NaN


In [3]:
df.isnull().sum()

title          0
text           0
stars          0
label          0
language    7499
dtype: int64

In [4]:
text = df["text"]
df.shape
df.head()

,title,text,stars,label,language
0,gAAAAABnUH4NgkTQ9eyZpcZr0JpVZKT2A_-T3BUnPBPZps...,Sauce vipo infecte piquante sans aucun goût ni...,2,0,NaN
1,gAAAAABnUH4NyGLVcUk0aT-6YySNpN-Atb905EoDedSWV2...,"La cuisine est médiocre, la sauce Vipo est san...",1,0,NaN
2,gAAAAABnUH4NDFwRDSycnVfF_qiCYlFhB60ebSobqCmoWh...,الاجواء و الاضاءة روعة\n النظافة في القمة\n ال...,5,1,NaN
3,gAAAAABnUH4NRqzylGPMu-ODANAHON7KKDiz4uJhnHJUfQ...,"J'ai vraiment aimé cette place, le service est...",4,1,"{""choices"":[""French"",""Darija""]}"
4,gAAAAABnUH4NiAgbqy6d_5sxsXSJ5Ly_TOLP2DMYqka9_4...,"Cozy place, good burgers for the price u pay, ...",4,1,English


In [5]:
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Remove punctuation
    # text = text.translate(str.maketrans('', '', string.punctuation))
    
    
    # Optional: Further cleaning, e.g., removing stopwords or stemming can be added here

    return text

def replace_with_synonyms(text):
    words = text.split()
    new_words = []
    for word in words:
        synonyms = wordnet.synsets(word, lang="fra")  
        if synonyms:
            synonym = random.choice(synonyms).lemmas("fra")[0].name()
            new_words.append(synonym if synonym != word else word)
        else:
            new_words.append(word)
    return " ".join(new_words)

def is_french(text):
    try:
        if not text.strip(): 
            return False
        return detect(text) == "fr"
    except LangDetectException:
        return False

rows_to_duplicate = df[(df["label"] == 0) & (df["text"].apply(is_french))]
new_rows = []
for _, row in rows_to_duplicate.iterrows():
    new_text = replace_with_synonyms(row["text"])
    print(row["text"])
    print(new_text)
    new_rows.append({"text": new_text, "label": row["label"], "title": row["title"]+"1", "stars" : row["stars"], "label": row["label"], "language": row["language"]})

df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)

# TF-IDF Vectorizer with preprocessing
vectorizer = TfidfVectorizer(stop_words=None, ngram_range=(1, 2))

# Create a pipeline
pipeline = Pipeline([
    ('preprocessor', FunctionTransformer(lambda x: [preprocess_text(t) for t in x], validate=False)),
    ('tfidf', vectorizer)
])

# Sample text data
texts = df["text"]

# Apply the pipeline
X_tfidf = pipeline.fit_transform(texts)

# Print the TF-IDF matrix
# print(X_tfidf.toarray())
# print(vectorizer.get_feature_names_out())


Sauce vipo infecte piquante sans aucun goût ni saveur tacos manque de saveur ou est le goût du fromage ? Inexistable, dommage la sauce est légèrement sucrée c est une sauce ketchup au piment je vous demande de revisiter votre recette bonne chance
sauce vipo infecte piquante baie_de_sureau aucun goût ni goût tacos défaut appliquer aromatiser OU est domaine goût du fromage ? Inexistable, dommage La aubaine est compétence sucrée c EST an sauce ketchup AU piment Moi vous créance appliquer revisiter votre formule bonne aubaine
La cuisine est médiocre, la sauce Vipo est sans vouloir paraître désagréable n'est pas appétissante du tout, trop sucré on dirait littéralement un mélange de ketchup et de piment, honnêtement je suis déçue, car pour le prix je suis censée avoir quelque chose de mangeable, ce qui n'était pas du tout le cas, je ne recommande pas ce fastfood, qui d'ailleurs n'est pas fast pour le coup, car il faut attendre un paquet de temps pour être servie.
Louisiane art_culinaire Est 

In [6]:
df.shape
text = df["text"]

In [7]:
from sklearn.model_selection import train_test_split, cross_val_score
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(text, df["label"], test_size=0.2, random_state=42)


In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.naive_bayes import MultinomialNB 

models = {
    # "Random Forest": RandomForestClassifier(),
    "SVM": SVC(),
    # "NB": MultinomialNB()
}

In [16]:

from sklearn.metrics import accuracy_score, classification_report

# Loop through models and train
for model_name, model in models.items():
    print(f"Training {model_name}...")
    
    # Create a pipeline with preprocessing, vectorization, and model
    def preprocess_texts(texts):
        return [preprocess_text(t) for t in texts]
    model_pipeline = Pipeline([
        ('preprocessor', FunctionTransformer(preprocess_texts, validate=False)),
        ('tfidf', vectorizer),
        ('classifier', model)
    ])
    
    # Train the model
    model_pipeline.fit(X_train, y_train)
    
    # Predict and evaluate
    y_pred = model_pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"{model_name} Accuracy: {accuracy}")
    print(f"Classification Report for {model_name}:\n", classification_report(y_test, y_pred))


Training SVM...
SVM Accuracy: 0.9219119734973971
Classification Report for SVM:
               precision    recall  f1-score   support

           0       0.92      0.90      0.91       978
           1       0.92      0.94      0.93      1135

    accuracy                           0.92      2113
   macro avg       0.92      0.92      0.92      2113
weighted avg       0.92      0.92      0.92      2113



In [18]:
import joblib

joblib.dump(model_pipeline, './models/svm.pkl')


['./models/svm.pkl']

In [43]:
    from sklearn.model_selection import GridSearchCV
    print(f"Performing Cross-Validation for SVM")

    param_grid = {
    'classifier__C': [0.1, 1, 10],  # Regularization parameter
    'classifier__kernel': ['linear', 'rbf'],  # Kernel types
    'classifier__gamma': ['scale', 'auto']  # Kernel coefficient for 'rbf'
    }

    model = SVC()

    # Create a pipeline with preprocessing, vectorization, and model
    model_pipeline = Pipeline([
        ('preprocessor', FunctionTransformer(lambda x: [preprocess_text(t) for t in x], validate=False)),
        ('tfidf', vectorizer),
        ('classifier', model)
    ])

    grid_search = GridSearchCV(model_pipeline, param_grid, cv=5, scoring='f1_macro', verbose=1, n_jobs=-1)

    grid_search.fit(X_train, y_train)

    # Perform cross-validation
    scores = cross_val_score(grid_search, X_train, y_train, cv=5, scoring='f1_weighted')  
    
    print(f"{model_name} Cross-Validation Accuracy Scores: {scores}")
    print(f"Mean Accuracy for {model_name}: {scores.mean():.4f}")
    print(f"Standard Deviation for {model_name}: {scores.std():.4f}\n")



Performing Cross-Validation for SVM
Fitting 5 folds for each of 12 candidates, totalling 60 fits


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
SVM Cross-Validation Accuracy Scores: [0.90033637 0.90842434 0.91527242 0.88986211 0.8855054 ]
Mean Accuracy for SVM: 0.8999
Standard Deviation for SVM: 0.0111



In [75]:
grid_search.best_params_

{'classifier__C': 1,
 'classifier__gamma': 'scale',
 'classifier__kernel': 'linear'}

In [82]:
test = pd.read_csv('./data/test_food.csv', index_col=0).reset_index()
print(test[test['review_text'].isna()])
test = test.dropna(subset=['review_text'])
test["Sentiment"] = model_pipeline.predict(test["review_text"])
out = test[["iD", "Sentiment"]]
out.to_csv('test_food_sentiment_submission.csv', index=False)

                                       iD review_text
646  04c109cc-61b5-4c47-a0f2-5f5e193787c1         NaN
